In [1]:
# -*- coding = utf-8 -*-
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dense

In [2]:
resnet = ResNet50(include_top=False, pooling='avg')
model = Sequential()
model.add(resnet)
model.add(Dense(1))
model.layers[0].trainable = False
model.add(Dense(2, activation='softmax'))
print(model.summary())

24182784/94765736 [======>.......................] - ETA: 14s

KeyboardInterrupt: 

In [3]:
num_skipped = 0
for folder_name in ("beauty", "ugly"):
    folder_path = os.path.join("./train", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 0 images


In [26]:
batch_size = 32
img_height = 180
img_width = 180
epochs = 30
train_dir = './train'
val_dir = './val'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  label_mode='categorical',  
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  val_dir,
  label_mode='categorical', 
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


Found 499 files belonging to 2 classes.
Using 400 files for training.
Found 160 files belonging to 2 classes.
Using 32 files for validation.


In [79]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=["accuracy"],
)
model.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/30
13/13 [==============================] - 13s 992ms/step - loss: 0.2632 - accuracy: 0.5450 - val_loss: 0.2813 - val_accuracy: 0.5000
Epoch 2/30
13/13 [==============================] - 12s 943ms/step - loss: 0.2291 - accuracy: 0.6275 - val_loss: 0.2107 - val_accuracy: 0.5938
Epoch 3/30
13/13 [==============================] - 12s 941ms/step - loss: 0.1979 - accuracy: 0.6975 - val_loss: 0.2130 - val_accuracy: 0.5000
Epoch 4/30
13/13 [==============================] - 12s 939ms/step - loss: 0.1714 - accuracy: 0.7675 - val_loss: 0.2309 - val_accuracy: 0.6562
Epoch 5/30
13/13 [==============================] - 12s 930ms/step - loss: 0.1552 - accuracy: 0.8050 - val_loss: 0.2241 - val_accuracy: 0.6250
Epoch 6/30
13/13 [==============================] - 12s 934ms/step - loss: 0.1439 - accuracy: 0.8375 - val_loss: 0.2337 - val_accuracy: 0.6250
Epoch 7/30
13/13 [==============================] - 12s 935ms/step - loss: 0.1357 - accuracy: 0.8300 - val_loss: 0.2474 - val_accuracy: 0.6250

In [80]:
model.save('./my_model') 

INFO:tensorflow:Assets written to: ./my_model\assets


In [2]:
new_model = tf.keras.models.load_model('./my_model')

In [3]:
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 2049      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 4         
Total params: 23,589,765
Trainable params: 23,536,645
Non-trainable params: 53,120
_________________________________________________________________


In [4]:
loss,acc = new_model.evaluate(val_ds, verbose=2)

NameError: name 'val_ds' is not defined

In [27]:
for image, label in val_ds.take(1):
    print(image.shape)
    print(new_model.predict(image))
    print(label)

(32, 180, 180, 3)
[[5.89156412e-02 9.41084385e-01]
 [2.47967035e-01 7.52032936e-01]
 [5.83380938e-01 4.16619033e-01]
 [8.95127416e-01 1.04872569e-01]
 [1.28636863e-02 9.87136245e-01]
 [2.62927022e-02 9.73707259e-01]
 [4.63626347e-02 9.53637362e-01]
 [9.88568544e-01 1.14315134e-02]
 [9.40114737e-01 5.98852299e-02]
 [6.11478329e-01 3.88521641e-01]
 [1.26235289e-02 9.87376451e-01]
 [8.75549670e-03 9.91244495e-01]
 [3.02862003e-02 9.69713807e-01]
 [5.55732346e-04 9.99444306e-01]
 [9.79422271e-01 2.05777418e-02]
 [8.46830428e-01 1.53169617e-01]
 [4.24230695e-01 5.75769365e-01]
 [8.38498294e-01 1.61501706e-01]
 [4.84334469e-01 5.15665472e-01]
 [9.91683722e-01 8.31623375e-03]
 [2.18958501e-02 9.78104174e-01]
 [1.03620039e-02 9.89637971e-01]
 [5.43078721e-01 4.56921220e-01]
 [9.94491816e-01 5.50817931e-03]
 [9.85482216e-01 1.45177767e-02]
 [9.95313883e-01 4.68609715e-03]
 [4.62079018e-01 5.37920952e-01]
 [8.71804297e-01 1.28195748e-01]
 [8.84448349e-01 1.15551606e-01]
 [9.55876783e-02 9.044123

In [32]:
img = cv2.imread('./201.jpg')

In [33]:
img = cv2.resize(img, (180, 180))
re_img = img.reshape([1] + list(img.shape))
print(re_img.shape)
new_model.predict(re_img)

(1, 180, 180, 3)


array([[0.927248  , 0.07275193]], dtype=float32)